<strong>LGBRanker (LightGBM Ranker)</strong>는 **순위화(Learning to Rank)** 문제를 해결하기 위한 **LightGBM 기반의 Gradient Boosting 모델**입니다. 아래는 LGBRanker에 대한 **개념**, **동작 원리**, **핵심 파라미터**, **적용 예시**, 그리고 **일반 분류/회귀 모델과의 차이점** 등을 중심으로 정리한 설명입니다.

---

# 🧠 LGBRanker (LightGBM Ranking) 개요

## 💡 학습 목적

LGBRanker는 단순히 값을 예측하는 것이 아니라, **아이템 간의 상대적인 순서를 예측**하는 데 중점을 둡니다.
즉, "이 문서가 더 좋은가?", "이 상품이 더 적절한가?" 같은 문제를 해결하는 데 특화되어 있습니다.

---

## 🔍 주요 사용 분야

| 분야            | 예시                    |
| ------------- | --------------------- |
| **검색 엔진**     | 문서 순서 정렬, 질의 응답 결과 랭킹 |
| **추천 시스템**    | 상품/동영상 등 사용자 관심 순서화   |
| **광고 클릭 예측**  | 광고별 클릭 확률에 따른 정렬      |
| **금융/리스크 평가** | 위험 우선순위 예측            |

---

# ⚙️ 모델 구조 및 동작 원리

LGBRanker는 **Gradient Boosted Decision Tree (GBDT)** 기반으로 구성되며, 기본적인 학습 방식은 LightGBM과 같지만, **손실 함수와 라벨 처리 방식이 다릅니다.**

## 💥 핵심 개념: Query Group

* **하나의 그룹 안에서는 순서를 비교할 수 있지만**, **그룹 간에는 비교 불가**
* 예: 검색 질의별 결과 → 각 질의가 하나의 query group

$$
\text{예시:}
\quad
\text{Group 1 (Query: 'lightgbm')} → [doc1, doc2, doc3]  
\quad
\text{Group 2 (Query: 'cnn')} → [doc4, doc5]
$$

---

## 🎯 주요 순위화 손실 함수

LGBRanker는 다음과 같은 손실 함수를 지원합니다:

| 이름                | 설명                              |
| ----------------- | ------------------------------- |
| `lambdarank`      | NDCG(순위 기반 평가) 최대화를 목표로 한 손실 함수 |
| `rank_xendcg`     | 빠르고 효율적인 NDCG 최적화 변형            |
| `lambdarank:ndcg` | 명시적으로 NDCG 사용                   |

이들은 모두 pairwise 혹은 listwise ranking에 기반한 **NDCG, MAP, DCG 등의 평가 지표**를 최적화합니다.

---

## 📌 주요 파라미터

| 파라미터         | 설명                                                    |
| ------------ | ----------------------------------------------------- |
| `objective`  | 보통 `lambdarank`, `rank_xendcg` 등 설정                   |
| `metric`     | `ndcg`, `map` 등 랭킹 품질 평가용                             |
| `group`      | 같은 쿼리 그룹별로 item 순위 평가                                 |
| `label_gain` | 라벨 등급별 가중치 설정 (예: relevance 3이 relevance 2보다 더 중요하도록) |

---

## 📊 입력 형태 예시

| query\_id | feature1 | feature2 | relevance\_label |
| --------- | -------- | -------- | ---------------- |
| 1         | 0.2      | 0.8      | 3                |
| 1         | 0.5      | 0.4      | 1                |
| 2         | 0.7      | 0.1      | 2                |
| 2         | 0.3      | 0.9      | 0                |

→ `group = [2, 2]`: 그룹 1에 2개, 그룹 2에 2개 쿼리

---

## 🧪 Python 예시 (LightGBM)

```python
import lightgbm as lgb

# 데이터셋 구성
train_data = lgb.Dataset(X_train, label=y_train, group=group_train)

# 파라미터 설정
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'learning_rate': 0.05,
    'num_leaves': 31
}

# 모델 훈련
model = lgb.train(params, train_data, num_boost_round=100)
```

---

# 🔁 회귀/분류와의 차이점

| 항목        | 회귀/분류 모델       | LGBRanker (순위화 모델)           |
| --------- | -------------- | ---------------------------- |
| 목적        | 값 예측 / 클래스 분류  | **상대적 순서 예측**                |
| 라벨 구조     | 개별 값 or 클래스    | **그룹 내 상대적 중요도 (relevance)** |
| 손실 함수     | MSE, Logloss 등 | **LambdaRank, NDCG 기반 손실**   |
| 입력 데이터 분리 | 없음             | **group 정보 필요**              |

---

# ✅ 장점 요약

* 정렬 기반 문제에서 **분류/회귀보다 더 직접적 최적화**
* **NDCG, MAP 등 실제 사용 지표와 정렬**
* 매우 빠른 학습 속도와 높은 확장성 (LightGBM 특성)

---

# ⚠️ 유의할 점

* `group` 정보 없이 사용하면 일반 분류기처럼 동작함 → **반드시 그룹 정보 포함**
* Label Gain, NDCG 기반 지표에 대해 명확한 이해가 필요
* 클래스 간 경계가 명확하지 않은 경우 성능 떨어질 수 있음

---

# 🧩 결론

**LGBRanker**는 단순 예측 모델이 아닌, \*\*"어떤 것이 더 중요하거나 앞에 와야 하는가"\*\*를 직접적으로 학습하는 모델로,
검색, 추천, 광고 등 **정렬이 중요한 실전 문제에서 매우 유용**합니다.

> ✅ 순위 평가 최적화
> ✅ 쿼리/그룹 단위의 정보 반영
> ✅ 대규모 데이터에 빠르게 학습 가능
